In [1]:
%%capture
!pip install --upgrade optuna_integration

In [2]:
import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import optuna.integration.lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.base import clone
from lightgbm import LGBMClassifier

SEED = 2024

In [3]:
DATA_DIR = '/kaggle/input/ml-olympiad-predicting-earthquake-damage'

train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
sample_sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [4]:
TARGET = 'damage_grade'
train[TARGET] = train[TARGET] - 1  # reversed in postprocessing

In [5]:
train = train.drop('building_id', axis=1)
test = test.drop('building_id', axis=1)

train = pd.get_dummies(train, dtype='int8')
test = pd.get_dummies(test, dtype='int8')

In [6]:
features = list(test.columns)
num_features = ['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families']
cat_features = [f for f in features if f not in num_features]

In [7]:
# LightGBM dataset
dtrain = lgb.Dataset(
    data=train[features],
    label=train[TARGET],
    feature_name=features,
    categorical_feature=cat_features)

In [8]:
base_params = {
    'objective': 'multiclass',
    'num_class': 3,
    'is_unbalance': True,
    'metric': 'multi_logloss',
    'learning_rate': 0.005,
    'boosting_type': 'gbdt',
    'force_row_wise': True,
    'verbosity': -1,
    'n_jobs': -1,
    'deterministic': True,
    'random_state': SEED
}

In [9]:
early_stopping = lgb.early_stopping(
    stopping_rounds=100,
    first_metric_only=True,
    verbose=False,
    min_delta=1e-4)

In [10]:
BUDGET = 60 * 60 * 1.5
NUM_FOLDS = 7

tuner = lgb.LightGBMTunerCV(
    time_budget=BUDGET,
    optuna_seed=SEED,
    params=base_params,
    train_set=dtrain,
    num_boost_round=5000,
    nfold=NUM_FOLDS,
    stratified=True,
    shuffle=True,
    seed=SEED,
    feature_name=features,
    categorical_feature=cat_features,
    callbacks=[early_stopping])

[I 2024-04-20 04:08:18,568] A new study created in memory with name: no-name-a578171c-6291-416f-abba-a5ed9c80705e


In [11]:
%%time
tuner.run()

min_child_samples, val_score: 0.832932:  60%|######    | 3/5 [02:03<01:22, 41.24s/it]

CPU times: user 1h 14min 44s, sys: 59min, total: 2h 13min 44s
Wall time: 1h 30min


In [12]:
print(f'Best score: {tuner.best_score:.5f}')
print(f'Best hyperparameters:')
for k, v in tuner.best_params.items():
    print(f'{k:20} - {v}')

Best score: 0.83293
Best hyperparameters:
objective            - multiclass
num_class            - 3
is_unbalance         - True
metric               - multi_logloss
learning_rate        - 0.005
boosting_type        - gbdt
force_row_wise       - True
verbosity            - -1
n_jobs               - -1
deterministic        - True
random_state         - 2024
feature_pre_filter   - False
lambda_l1            - 2.5538421574295447e-05
lambda_l2            - 2.8244787738425244e-07
num_leaves           - 6
feature_fraction     - 0.4
bagging_fraction     - 1.0
bagging_freq         - 0
min_child_samples    - 10


In [13]:
def comp_metric(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

def custom_cv(estimator, seed=SEED, verbose=True):
    X_test = test[features]
    
    oof_preds, test_preds = {}, {}
    scores = []

    cv = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=seed)
    for fold, (train_ids, val_ids) in enumerate(cv.split(train, train[TARGET])):
        X_train, y_train = train[features].iloc[train_ids], train[TARGET].iloc[train_ids]
        X_val, y_val = train[features].iloc[val_ids], train[TARGET].iloc[val_ids]
        
        model = clone(estimator)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            callbacks=[early_stopping])

        val_preds = model.predict(X_val)
        oof_preds.update(dict(zip(val_ids, val_preds)))
        test_preds[f'fold{fold}'] = model.predict(X_test)

        score = comp_metric(y_val, val_preds)
        scores.append(score)
        if verbose:
            print(f'Fold #{fold:>2}: {score:.5f} ({model.best_iteration_:>4} rounds)')
        _ = gc.collect()

    test_preds = pd.DataFrame.from_dict(test_preds)
    test_preds['mode'] = test_preds.mode(axis=1)[0].astype('int') # mode of fold-wise predictions
    
    oof_preds = pd.Series(oof_preds).sort_index()
    print(f'\nAvg score: {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    print(f'OOF score: {comp_metric(train[TARGET], oof_preds):.5f}\n')
    
    return oof_preds, test_preds

In [14]:
%%time
model = LGBMClassifier(**tuner.best_params, n_estimators=5000)
op, tp = custom_cv(model)

Fold # 0: 0.51662 (1226 rounds)
Fold # 1: 0.51117 ( 927 rounds)
Fold # 2: 0.47386 ( 983 rounds)
Fold # 3: 0.50348 ( 927 rounds)
Fold # 4: 0.49327 (1281 rounds)
Fold # 5: 0.51357 ( 876 rounds)
Fold # 6: 0.52943 (1147 rounds)

Avg score: 0.50591 +/- 0.01668
OOF score: 0.50647

CPU times: user 21.6 s, sys: 0 ns, total: 21.6 s
Wall time: 21.7 s


In [15]:
def create_submission_files(preds, notebook='01'):
    for col in preds.columns:
        sub = sample_sub.copy()
        # inverse transform for the target
        sub[TARGET] = (preds[col] + 1)
        sub.to_csv(f'nb{notebook}_{col}.csv', index=False)

In [16]:
create_submission_files(tp)